In [ ]:
import networkx as nx
import random
from torch_geometric.utils.convert import from_networkx

In [ ]:
num_nodes = 10 # for testing
num_edges = 20
ndd_percentage = 0.3
nodes = range(num_nodes)
g = nx.DiGraph()

node_types = [
    "NDD", # non-directed donors
    "PDP", # patient-donor pair
    "P", # pacient without a pair
]
node_type_distribution = [0.2, 0.4, 0.4]

# add nodes
for node in nodes:
    node_type = random.choices(population=node_types, weights=node_type_distribution)[0]
    g.add_node(node, type=node_type)

# DEBUG:
g.nodes[0]["type"]

In [ ]:
## add edges (no features, directed)
# TODO refactor
for edge_id in range(num_edges):
    src_node_id = random.choice(nodes)
    # avoid having patient nodes as source of edges
    while(g.nodes[src_node_id]["type"] == "P"):
        src_node_id = random.choice(nodes)
    dst_node_id = random.choice(nodes)
    # avoid self loops 
    # AND avoid NDD nodes as destination of edges
    while(src_node_id == dst_node_id or g.nodes[dst_node_id]["type"] == "NDD"):
        dst_node_id = random.choice(nodes)
    # add some random weight to the edge
    edge_weight = random.random()
    g.add_edge(src_node_id, dst_node_id, edge_weights=edge_weight)

In [ ]:
## Visualize graph

# color nodes according to their type
color_map = []
for node_id in g:
    node_type = g.nodes[node_id]["type"]
    if node_type == "NDD":
        color_map.append('lightgreen')
    elif node_type == 'PDP':
        color_map.append('lightblue')
    elif node_type == "P":
        color_map.append('red')
    else:
        raise ValueError(f'Type {node_type} is not valid.')
nx.draw(G=g, node_color=color_map, with_labels=True)

In [ ]:
# TODO solve instance (exact solution)

In [ ]:
# Convert the graph into PyTorch geometric
pyg_graph = from_networkx(g)
pyg_graph


In [ ]:
#  create a random solution
src_nodes = pyg_graph.edge_index[0].tolist()
dst_nodes = pyg_graph.edge_index[1].tolist()
# for each node, leave maximum of one edge leaving and maximum of one edge entering
src_seen = []
dst_seen = []
solution_edge_indexes = []
for i, src_node in enumerate(src_nodes):
    dst_node = dst_nodes[i]
    if src_node not in src_seen and dst_node not in dst_seen:
        solution_edge_indexes.append(i)
        dst_seen.append(dst_node)
        src_seen.append(src_node)
        # print(f"DEBUG i: {i}, src_node: {src_node}, dst_node: {dst_node}")


In [ ]:
## Visualize solution
posisions_dict = nx.spring_layout(g)
path_edges = [
    pyg_graph.edge_index[:, solution_edge_indexes[i]].tolist() for i in range(len(solution_edge_indexes))
]
# print(posisions_dict)
# print(path_edges)
# print(pyg_graph.edge_index)
nx.draw(G=g, pos=posisions_dict, node_color=color_map, with_labels=True)
nx.draw_networkx_edges(G=g, pos=posisions_dict, edgelist=path_edges, edge_color ='r', width=3)


In [ ]:
## evaluate solution

#  sum of all edge weights
total_weight_sum = sum(pyg_graph.edge_weights)
print(pyg_graph.edge_weights)
print(f"total_weight_sum: {total_weight_sum}")
# sum of all edge weights in solution
solution_weight_sum = sum(pyg_graph.edge_weights[solution_edge_indexes])
print(f"solution_weight_sum: {solution_weight_sum}")
# sum of all edge weights outside solution
# TODO consider only receiving nodes
print(f"not-solution weight_sum: {total_weight_sum - solution_weight_sum}")
# edge_weight percentage
print(f"solution_weight_sum/total_weight_sum: {solution_weight_sum/total_weight_sum}")